<a href="https://colab.research.google.com/github/nhhung1810/8-queens-SAT/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A SAT Approach for 8 Queens Problem

In [ ]:
## Preparation

In [ ]:
# A* solver


In [ ]:
!pip install python-sat
from pysat.formula import CNF
from pysat.solvers import Lingeling
from typing import List, Tuple

     |████████████████████████████████| 1.8 MB 10.3 MB/s 


## Requirement a: Formulate the problem by specifying the following points.

(Khi nào cài xong thì ghi)

## Requirement b: Write CNF clauses to describe restrictions required when Florence places a queen in the cell[3][3]

First of all, denote `b[x][y]` as "There is a queen placed at the cell (x, y)" and `-b[x][y]` as "It's invalid to place a queen at the cell (x, y)".
If Florence places a queen in the cell (3, 3) all other cells in the horizontal, vertical, main & sub diagonal line are invalid cells to place the next queen. Therefore:

$b[3][3] \implies -b[0][3] \wedge -b[1][3] \wedge -b[2][3] \wedge -b[4][3] \wedge -b[5][3] \wedge -b[6][3] \wedge -b[7][3] \wedge $

$-b[3][0] \wedge -b[3][1] \wedge -b[3][2] \wedge -b[3][4] \wedge -b[3][5] \wedge -b[3][6] \wedge -b[3][7] \wedge $

$-b[0][0] \wedge -b[1][1] \wedge -b[2][2] \wedge -b[4][4] \wedge -b[5][5] \wedge -b[6][6] \wedge -b[7][7] \wedge $

$-b[0][6] \wedge -b[1][5] \wedge -b[2][4] \wedge -b[4][2] \wedge -b[5][1] \wedge -b[6][0]$

Which equivalents to:

$-b[3][3] ∨ (-b[0][3] \wedge -b[1][3] \wedge -b[2][3] \wedge -b[4][3] \wedge -b[5][3] \wedge -b[6][3] \wedge -b[7][3] \wedge $

$-b[3][0] \wedge -b[3][1] \wedge -b[3][2] \wedge -b[3][4] \wedge -b[3][5] \wedge -b[3][6] \wedge -b[3][7] \wedge $

$-b[0][0] \wedge -b[1][1] \wedge -b[2][2] \wedge -b[4][4] \wedge -b[5][5] \wedge -b[6][6] \wedge -b[7][7] \wedge $

$-b[0][6] \wedge -b[1][5] \wedge -b[2][4] \wedge -b[4][2] \wedge -b[5][1] \wedge -b[6][0])$

Transforming into CNF clause:

$ (-b[3][3] \vee -b[0][3]) \wedge $

$ (-b[3][3] \vee -b[1][3]) \wedge $

$ (-b[3][3] \vee -b[2][3]) \wedge $

$ (-b[3][3] \vee -b[4][3]) \wedge $

$ (-b[3][3] \vee -b[5][3]) \wedge $

$ (-b[3][3] \vee -b[6][3]) \wedge $

$ (-b[3][3] \vee -b[7][3]) \wedge $

$ (-b[3][3] \vee -b[3][0]) \wedge $

$ (-b[3][3] \vee -b[3][1]) \wedge $

$ (-b[3][3] \vee -b[3][2]) \wedge $

$ (-b[3][3] \vee -b[3][4]) \wedge $

$ (-b[3][3] \vee -b[3][5]) \wedge $

$ (-b[3][3] \vee -b[3][6]) \wedge $

$ (-b[3][3] \vee -b[3][7]) \wedge $

$ (-b[3][3] \vee -b[0][0]) \wedge $

$ (-b[3][3] \vee -b[1][1]) \wedge $

$ (-b[3][3] \vee -b[2][2]) \wedge $

$ (-b[3][3] \vee -b[4][4]) \wedge $

$ (-b[3][3] \vee -b[5][5]) \wedge $

$ (-b[3][3] \vee -b[6][6]) \wedge $

$ (-b[3][3] \vee -b[7][7]) \wedge $

$ (-b[3][3] \vee -b[0][6]) \wedge $

$ (-b[3][3] \vee -b[1][5]) \wedge $

$ (-b[3][3] \vee -b[2][4]) \wedge $

$ (-b[3][3] \vee -b[4][2]) \wedge $

$ (-b[3][3] \vee -b[5][1]) \wedge $

$ (-b[3][3] \vee -b[6][0]) $



## Requirement c: Create a function that returns the expected CNF set

In [ ]:
def encode_coor (x: int, y: int, size: int) -> int:
  return x * size + y + 1

def decode_coor (v: int, size: int) -> Tuple[int, int]:
  v -= 1
  return v // size, v % size

In [ ]:
def restriction_at (x: int, y: int, size: int) -> List[List[int]]:
  if x < 0 or size <= x or y < 0 or size <= y:
    return []

  res = []
  # Horizontal
  for i in range(size):
    if i == x:
      continue
    res.append([-1 * encode_coor(x, y, size), -1 * encode_coor(i, y, size)])

  # Vertical
  for j in range(size):
    if j == y:
      continue
    res.append([-1 * encode_coor(x, y, size), -1 * encode_coor(x, j, size)])

  # Main diagonal
  for i in range(size):
    j = i - (x - y)
    if j < 0 or size <= j or (i == x and j == y):
      continue

    res.append([-1 * encode_coor(x, y, size), -1 * encode_coor(i, j, size)])

  # Sub diagonal
  for i in range(size):
    j = (x + y) - i
    if j < 0 or size <= j or (i == x and j == y):
      continue

    res.append([-1 * encode_coor(x, y, size), -1 * encode_coor(i, j, size)])

  return res

In [ ]:
def restrictions (level: int, size: int, selected: List[int] = [], excluded: List[int] = []) -> List[List[int]]:
  res = []
  for i in range(size):
    for j in range(size):
      res += restriction_at(i, j, size)

  if level == 1:
    for i in range(size):
      res.append([encode_coor(i, j, size) for j in range(size)])
  elif level == 2:
    candidates = []
    for v in range(1, 1 + size * size):
      if v in excluded:
        res.append([-1 * v])
      else:
        if v in selected:
          res.append([v])
        else:
          candidates.append(v)
    if len(candidates) > 0:
      res.append(candidates)

  return res

# restrictions(2, 4, [3, 4], [1, 2])

In [ ]:
def solve (level: int = 1, size: int = 8) -> List[int]:
  if level == 1:
    res = restrictions(1, size)
    formula = CNF()
    for r in res:
      formula.append(r)

    l = Lingeling(bootstrap_with=formula.clauses, with_proof=True)
    if l.solve() == False:
      print("Failed, proof:", l.get_proof())
      return []
    else:
      print("Success")
      result = []
      for v in l.get_model():
        if v > 0:
          result.append(v)
      
      return result

  elif level == 2:
    result = []
    exclude = [[] for _ in range(size + 1)]
    cnt = 0
    while len(result) < size:
      # print(result, exclude)
      res = restrictions(2, size, result, exclude[len(result)])
      formula = CNF()
      for r in res:
        formula.append(r)

      l = Lingeling(bootstrap_with=formula.clauses, with_proof=True)
      if l.solve() == False:
        if len(result) == 0:
          print("Failed")
          return []
        print("Remove latest queen")
        exclude[len(result)] = []
        if len(result) > 0:
          exclude[len(result) - 1].append(result[-1])
          
        result = result[:-1]
      else:
        found = False
        for v in l.get_model():
          if v > 0 and v not in result:
            found = True
            result.append(v)
            print(f"Found the {len(result)}th queen at {str(decode_coor(v, size))}")

        if not found:
          print("wtf???")
          print(result, exclude)
          print(res)
          print(l.get_model())
          return []

    print("Success")
    return result

  else:
    print("Invalid level")
    return []

size = 8
res = solve(level = 1, size = size)
print([decode_coor(v, size) for v in res])
res = solve(level = 2, size = size)
print([decode_coor(v, size) for v in res])

Streaming output truncated to the last 5000 lines.
Remove latest queen
Found the 6th queen at (5, 3)
Found the 7th queen at (2, 4)
Remove latest queen
Remove latest queen
Found the 6th queen at (7, 3)
Found the 7th queen at (2, 4)
Remove latest queen
Found the 7th queen at (5, 7)
Remove latest queen
Remove latest queen
Remove latest queen
Found the 5th queen at (7, 5)
Found the 6th queen at (2, 4)
Remove latest queen
Remove latest queen
Remove latest queen
Found the 4th queen at (5, 4)
Found the 5th queen at (3, 5)
Found the 6th queen at (7, 3)
Remove latest queen
Remove latest queen
Found the 5th queen at (7, 3)
Found the 6th queen at (3, 5)
Remove latest queen
Found the 6th queen at (2, 5)
Remove latest queen
Found the 6th queen at (2, 6)
Remove latest queen
Remove latest queen
Found the 5th queen at (2, 5)
Found the 6th queen at (7, 3)
Remove latest queen
Found the 6th queen at (3, 7)
Remove latest queen
Remove latest queen
Found the 5th queen at (2, 6)
Found the 6th queen at (7, 3)